In [24]:
from CollectMaPSAs import *
import ROOT
import cPickle
import matplotlib.pyplot as plt
from scipy.special import erfc
from scipy.special import erf

In [25]:
mapsa_names = ["HPK44_1"]

In [26]:
mapsas = []
for m in mapsa_names:
    # Read MaPSA object from file, if available
    fname = 'pickles/'+m+'.pkl'
    if os.path.isfile(fname):
        print("Loading MaPSA " + m)
        mapsa = cPickle.load(open(fname,'rb'))
    else: # Create it
        print("MaPSA " + m +" has no pickle")

    mapsas += [mapsa]

Loading MaPSA HPK44_1


In [27]:
whichcurve = "CAL_posttrim"

chip = mapsas[0].mpa_chips[0]
        
if whichcurve == "CAL_pretrim":
    curves_all = chip.CALS_pretrim.transpose()[0:256]
    rms_fit_all = chip.pixels["CAL_RMS_pretrim"]
    mean_fit_all = chip.pixels["CAL_Mean_pretrim"]
elif whichcurve == "CAL_posttrim":
    curves_all = chip.CALS.transpose()[0:256]
    rms_fit_all = chip.pixels["CAL_RMS"]
    mean_fit_all = chip.pixels["CAL_Mean"]
elif whichcurve == "Bump":
    curves_all = chip.BumpS.transpose()[0:256]
    rms_fit_all = chip.pixels["Bump_RMS"]
    mean_fit_all = chip.pixels["Bump_Mean"]

In [28]:
def derivative_root(curve):
    h = ROOT.TH1D('h','h',256,0,256)
    y = np.diff(curve) 
    for i in range(0,255):
        h.SetBinContent(i+1,y[i])
    return [h.GetMean(),h.GetRMS()]

In [29]:
def derivative(curve):
    y = np.diff(curve) 
    tot = 0.
    for i,x in enumerate(y):
        tot += (i+0.5)*x
    mean = tot/sum(y) 
    
    sigma2 = 0.
    for i,x in enumerate(y):
        sigma2 += x*(i+0.5-mean)**2
        
    sigma = np.sqrt(sigma2/sum(y))
    return [mean, sigma]

In [30]:
curves = curves_all
mean_fit = mean_fit_all
rms_fit = rms_fit_all

In [ ]:
mean_der = []
rms_der = []

for i in range(0,curves.shape[1]):
    out = derivative(curves[i])
    mean_der += [out[0]]
    rms_der += [out[1]]
    
    if i<10:
        fig, ax = plt.subplots()
        plt.scatter(np.linspace(0,255,256),curves[i],label='data')

        curve_fit = [500*(1+erf((x-mean_fit[i])/rms_fit[i])) for x in np.linspace(0,255,512)]
        # NB fit is offset by 1 in DAC
        plt.plot(np.linspace(1,256,512),curve_fit,label='fit')
        
        curve_der = [500*(1+erf((x-out[0])/(np.sqrt(2)*out[1]))) for x in np.linspace(0,255,512)]
        plt.plot(np.linspace(0,255,512),curve_der,label='derivative')
        
        ax.set_xlim(0,2*mean_fit[i])
        plt.legend(loc='upper left',prop={'size': 12});

/cvmfs/sft.cern.ch/lcg/views/LCG_96/x86_64-centos7-gcc8-opt/lib/python2.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in sqrt
  if sys.path[0] == '':


In [ ]:
plt.hist(rms_fit/np.sqrt(2),histtype='step',bins=np.linspace(0,10,100));
plt.hist(rms_der,histtype='step',bins=np.linspace(0,10,100));
plt.legend(['fit','derivative'],loc='upper right',prop={'size': 12});

In [ ]:
plt.hist(mean_fit,histtype='step',bins=np.linspace(0,25,100));
plt.hist(mean_der,histtype='step',bins=np.linspace(0,25,100));
plt.legend(['fit','derivative'],loc='upper right',prop={'size': 12})

In [ ]:
plt.hist(rms_der-rms_fit,bins=np.linspace(-5,5,100),histtype='step');

In [ ]:
plt.hist(mean_der-mean_fit,bins=np.linspace(-5,5,100),histtype='step');